In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/credits.csv
/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/keywords.csv
/kaggle/input/the-movies-dataset/ratings_small.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/links.csv


In [7]:
def str_to_set(x):
    return set(item['name'] for item in eval(x))
    
meta = pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv')
meta = meta[['id', 'original_title', 'original_language', 'genres']]
meta = meta.rename(columns={'id':'movieId',
                            'original_title':'title',
                            'original_language':'language'})

meta = meta.loc[meta['language'] == 'en', :]
meta.movieId = pd.to_numeric(meta.movieId)

meta.genres = meta.genres.apply(str_to_set)
meta

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,movieId,title,language,genres
0,862,Toy Story,en,"{Comedy, Family, Animation}"
1,8844,Jumanji,en,"{Adventure, Family, Fantasy}"
2,15602,Grumpier Old Men,en,"{Comedy, Romance}"
3,31357,Waiting to Exhale,en,"{Comedy, Drama, Romance}"
4,11862,Father of the Bride Part II,en,{Comedy}
...,...,...,...,...
45459,222848,Caged Heat 3000,en,{Science Fiction}
45460,30840,Robin Hood,en,"{Drama, Action, Romance}"
45463,67758,Betrayal,en,"{Thriller, Action, Drama}"
45464,227506,Satana likuyushchiy,en,{}


In [8]:
keywords = pd.read_csv('/kaggle/input/the-movies-dataset/keywords.csv')
keywords.id = pd.to_numeric(keywords.id)
keywords = keywords.rename(columns={'id':'movieId'})
keywords.keywords = keywords.keywords.apply(str_to_set)
keywords

,movieId,keywords
0,862,"{boy next door, new toy, friends, boy, toy, to..."
1,8844,"{recluse, based on children's book, giant inse..."
2,15602,"{old men, fishing, best friend, duringcreditss..."
3,31357,"{single mother, interracial relationship, chic..."
4,11862,"{aging, baby, confidence, pregnancy, gynecolog..."
...,...,...
46414,439050,{tragic love}
46415,111109,"{play, artist, pinoy}"
46416,67758,{}
46417,227506,{}


In [9]:
meta = pd.merge(meta, keywords, on='movieId', how='inner')
meta

,movieId,title,language,genres,keywords
0,862,Toy Story,en,"{Comedy, Family, Animation}","{boy next door, new toy, friends, boy, toy, to..."
1,8844,Jumanji,en,"{Adventure, Family, Fantasy}","{recluse, based on children's book, giant inse..."
2,15602,Grumpier Old Men,en,"{Comedy, Romance}","{old men, fishing, best friend, duringcreditss..."
3,31357,Waiting to Exhale,en,"{Comedy, Drama, Romance}","{single mother, interracial relationship, chic..."
4,11862,Father of the Bride Part II,en,{Comedy},"{aging, baby, confidence, pregnancy, gynecolog..."
...,...,...,...,...,...
32847,222848,Caged Heat 3000,en,{Science Fiction},{}
32848,30840,Robin Hood,en,"{Drama, Action, Romance}",{}
32849,67758,Betrayal,en,"{Thriller, Action, Drama}",{}
32850,227506,Satana likuyushchiy,en,{},{}


In [10]:
dk = meta.loc[meta.title == 'The Dark Knight', :].iloc[0]
dkr = meta.loc[meta.title == 'The Dark Knight Rises', :].iloc[0]
toy = meta.loc[meta.title == 'Toy Story', :].iloc[0]
fk = meta.loc[meta.title == 'Prom Night', :].iloc[0]



dk_set = dk.genres | dk.keywords
dkr_set = dkr.genres | dkr.keywords
toy_set = toy.genres | toy.keywords
fk_set = fk.genres | fk.keywords

def jaccard_similarity(s1, s2):
    if len(s1 | s2) == 0:
        return 0
    return len(s1 & s2) / len(s1 | s2)

jaccard_similarity(dk_set, dkr_set)
jaccard_similarity(dk_set, toy_set)
jaccard_similarity(dk_set, fk_set)

0.022222222222222223

In [13]:
ratings = pd.read_csv('/kaggle/input/the-movies-dataset/ratings_small.csv')
ratings = pd.merge(ratings, meta[['movieId', 'title']], on='movieId', how='inner')
ratings

,userId,movieId,rating,timestamp,title
0,1,1371,2.5,1260759135,Rocky III
1,4,1371,4.0,949810302,Rocky III
2,7,1371,3.0,851869160,Rocky III
3,19,1371,4.0,855193404,Rocky III
4,21,1371,3.0,853852263,Rocky III
...,...,...,...,...,...
33295,652,127728,5.0,1439586990,8:46
33296,652,129009,4.0,1442690827,Love Is a Ball
33297,653,2103,3.0,948161066,Solaris
33298,659,167,4.0,836137550,K-PAX


In [15]:
matrix = ratings.pivot_table(index='userId', columns='title', values='rating')
matrix

title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 + 1,12 Angry Men,1408,...,Young and Innocent,Zaat,Zabriskie Point,Zapped Again!,Zardoz,Zodiac,eXistenZ,xXx,¡Three Amigos!,Мой сводный брат Франкенштейн
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
def pearson_similarity(u1, u2):
    u1_c = u1 - u1.mean()
    u2_c = u2 - u2.mean()
    denom = ((u1_c ** 2).sum() * (u2_c ** 2).sum()) ** 0.5
    if denom == 0:
        return 0
    else:
        return (u1_c * u2_c).sum() / denom
    

In [17]:
dk_rating = matrix['The Dark Knight']
pk_rating = matrix['Prom Night']
pearson_similarity(dk_rating, pk_rating)

0.773565934694095

In [20]:
def find_similar_movies(input_title, n, alpha):
    input_meta = meta.loc[meta['title'] == input_title].iloc[0]
    input_set = input_meta.genres | input_meta.keywords
    
    result = []
    
    for this_title in matrix.columns:
        if this_title == input_title:
            continue
        
        this_meta = meta.loc[meta['title'] == this_title].iloc[0]
        this_set = this_meta.genres | this_meta.keywords
        
        jaccard = jaccard_similarity(this_set, input_set)
        pearson = pearson_similarity(matrix[this_title], matrix[input_title])
        
        score = alpha * pearson + (1 - alpha) * jaccard
        result.append((this_title, pearson, jaccard, score))
    
    result.sort(key=lambda r: r[3], reverse=True)
    
    return result[:n]

In [21]:
res = find_similar_movies("The Dark Knight", 10, 0.5)
res

[('Wild Wild West', 0.773565934694095, 0.03225806451612903, 0.402911999605112),
 ('Prom Night', 0.773565934694095, 0.022222222222222223, 0.3978940784581586),
 ('Ludwig', 0.4727180852203643, 0.03333333333333333, 0.2530257092768488),
 ('Yamakasi - Les samouraïs des temps modernes',
  0.3771445116444287,
  0.125,
  0.2510722558222144),
 ('El asaltante', 0.44694920671214383, 0.04, 0.2434746033560719),
 ('Midnight in the Garden of Good and Evil',
  0.37384099242466423,
  0.1111111111111111,
  0.24247605176788767),
 ('Sneakers', 0.41582996031224023, 0.06896551724137931, 0.24239773877680976),
 ('Topaz', 0.3771445116444286, 0.10344827586206896, 0.24029639375324877),
 ('Godzilla', 0.4131411129338466, 0.05263157894736842, 0.2328863459406075),
 ('Bushwhacked', 0.39608096134212234, 0.06451612903225806, 0.2302985451871902)]

In [25]:
pd.DataFrame(res, columns=['title', 'jaccard', 'pearson', 'score'])

,title,jaccard,pearson,score
0,Wild Wild West,0.773566,0.032258,0.402912
1,Prom Night,0.773566,0.022222,0.397894
2,Ludwig,0.472718,0.033333,0.253026
3,Yamakasi - Les samouraïs des temps modernes,0.377145,0.125000,0.251072
4,El asaltante,0.446949,0.040000,0.243475
5,Midnight in the Garden of Good and Evil,0.373841,0.111111,0.242476
6,Sneakers,0.415830,0.068966,0.242398
7,Topaz,0.377145,0.103448,0.240296
8,Godzilla,0.413141,0.052632,0.232886
9,Bushwhacked,0.396081,0.064516,0.230299


In [26]:
res2 = find_similar_movies("Rocky III", 10, 0.5)
res2

[('Rocky V', 0.29715706440249456, 0.2727272727272727, 0.28494216856488364),
 ('Rocky II', 0.18271514219927637, 0.3333333333333333, 0.25802423776630484),
 ('Rocky Balboa',
  0.15268056083149895,
  0.35714285714285715,
  0.2549117089871781),
 ('Rocky IV', 0.1173537655037235, 0.34615384615384615, 0.23175380582878483),
 ('About Schmidt', 0.2952635163807467, 0.04, 0.16763175819037335),
 ('54', 0.2930602872142698, 0.041666666666666664, 0.16736347694046824),
 ('Cinderella Man',
  0.03037323884068473,
  0.2962962962962963,
  0.1633347675684905),
 ('Michael', 0.2871399771827741, 0.038461538461538464, 0.1628007578221563),
 ("I'll Sleep When I'm Dead",
  0.22365882172607585,
  0.09090909090909091,
  0.15728395631758338),
 ('Touch of Evil',
  0.2800752969801283,
  0.034482758620689655,
  0.15727902780040898)]

In [ ]:
pd.DataFrame(res2, columns=['title', 'jaccard', 'pearson', 'score'])